# Image analysis with kluster.ai API

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kluster-ai/klusterai-cookbook/blob/main/examples/sentiment-analysis-api.ipynb)

AI models can be used to perform image analysis tasks, in which you feed the model an image and request it to extract meaningful information.

This tutorial runs through a notebook where you'll learn how to use the <a href="https://kluster.ai/" target="_blank">kluster.ai</a> batch API to run image analysis on different images using both Gemma 3 27B and Qwen 2.5 7B.

The example uses three separate images. For each image, we will ask the models to fetch a specific feature, and compare how they respond:

1. A <a href="https://github.com/kluster-ai/docs/blob/main/images/tutorials/balls-image.jpg?raw=true" target="_blank">Newtown's cradle</a>, we will ask what is the device's name and how many balls are in the image (5)
2. <a href="https://github.com/kluster-ai/docs/blob/main/images/tutorials/eggs-image.jpg?raw=true" target="_blank">Eggs</a> of different colors, we will ask how many total eggs and per color (10 total, 8 brown and 2 white)
3. <a href="https://github.com/kluster-ai/docs/blob/main/images/tutorials/music-image.jpeg?raw=true" target="_blank">Hand written musical note</a>, we will ask what is the sequence of notes (C, D, E and F)

You can adapt this example by using your own images or request. With this approach, you can effortlessly process images of any scale, big or small, and obtain image analysis powered by a state-of-the-art language model.

## Prerequisites

Before getting started, ensure you have the following:

- **A kluster.ai account** - sign up on the <a href="https://platform.kluster.ai/signup" target="_blank">kluster.ai platform</a> if you don't have one
- **A kluster.ai API key** - after signing in, go to the <a href="https://platform.kluster.ai/apikeys" target="_blank">**API Keys**</a> section and create a new key. For detailed instructions, check out the <a href="/get-started/get-api-key/" target="_blank">Get an API key</a> guide

## Setup

In this notebook, we'll use Python's `getpass` module to input the key safely. After execution, please provide your unique kluster.ai API key (ensure no spaces).

In [1]:
from getpass import getpass

api_key = getpass("Enter your kluster.ai API key: ")

Enter your kluster.ai API key:  ········


Next, ensure you've installed OpenAI Python library:

In [2]:
pip install -q openai

Note: you may need to restart the kernel to use updated packages.


With the OpenAI Python library installed, we import the necessary dependencies for the tutorial:

In [3]:
from openai import OpenAI

import pandas as pd
import time
import json
import os
from IPython.display import clear_output, display

And then, initialize the `client` by pointing it to the kluster.ai endpoint, and passing your API key.

In [4]:
# Set up the client
client = OpenAI(
    base_url="https://api.kluster.ai/v1",
    api_key=api_key,
)

## Get the data

There are two main ways to proceed when working with images:

1. You can provide the the raw image file as a URL from source, for example, GitHub
2. You can provide Base64 encoded. These are typically represented with a blob of text, which starts with `data:image/png;base64,ENCODING_DATA_HERE...`

With both metholodogies, image data needs to be provided as an object in the content array with the following format:

```
...
{"type": "image_url", "image_url": {"url": IMAGE_DATA}}
...
```

Just replace `IMAGE_DATA` with either the URL with the raw image file, or the Base64 encoded image. This tutorial uses the the URL of the images uploaded to GitHub:


In [5]:
base_url = (
    "https://github.com/kluster-ai/docs/blob/main/images/tutorials/IMAGE_NAME?raw=true"
)

# Newton's cradle image
# Expected answer: Newton's cradle, 5 balls
image1 = "balls-image.jpg"
image1_url = base_url.replace("IMAGE_NAME", image1)
print(image1_url)


# Eggs
# Expected answer: 10 eggs, 8 brown and 2 white
image2 = "eggs-image.jpg"
image2_url = base_url.replace("IMAGE_NAME", image2)
print(image2_url)

# Musical note
# Expected answer: notes are, in order, C, D, E and F
image3 = "music-image.jpeg"
image3_url = base_url.replace("IMAGE_NAME", image3)
print(image3_url)


images = [image1, image2, image3]

https://github.com/kluster-ai/docs/blob/main/images/tutorials/balls-image.jpg?raw=true
https://github.com/kluster-ai/docs/blob/main/images/tutorials/eggs-image.jpg?raw=true
https://github.com/kluster-ai/docs/blob/main/images/tutorials/music-image.jpeg?raw=true


## Perform batch inference

To execute the batch inference job, we'll take the following steps:

1. **Create the batch job file** - we'll generate a JSON lines file with the desired requests to be processed by the model
2. **Upload the batch job file** - once it is ready, we'll upload it to the <a href="https://platform.kluster.ai/signup" target="_blank">kluster.ai platform</a> using the API, where it will be processed. We'll receive a unique ID associated with our file
3. **Start the batch job** - after the file is uploaded, we'll initiate the job to process the uploaded data, using the file ID obtained before
4. **Monitor job progress** - (optional) track the status of the batch job to ensure it has been successfully completed
5. **Retrieve results** - once the job has completed execution, we can access and process the resultant data

This notebook is prepared for you to follow along. Run the cells below to watch it all come together.

### Create the batch input file

This example uses two models that are more oriented to image vision/analysis: `google/gemma-3-27b-it` and `Qwen/Qwen2.5-VL-7B-Instruct`. Others model might not support providing images.

In addition, please refer to the <a href="/get-started/start-building/batch/#supported-models" target="_blank">Supported models</a> section for a list of the models we support.

The following snippets prepare the JSONL file, where each line represents a different request. Note that each separate batch request can have its own model. Also, we are using a temperature of `0.5` but feel free to change it and play around with the different outcomes.


In [41]:
# Prompt based on image
SYSTEM_PROMPTS = {
    "balls-image.jpg": """
    You are a helpful assistant that analyzes image content.
    Tell me the device depicted in the image, and how many balls it has.
    """,
    "eggs-image.jpg": """
    You are a helpful assistant that analyzes image content.
    Count how many eggs are in total, and how many brown and white eggs separately.
    """,
    "music-image.jpeg": """
    You are a helpful assistant that analyzes image content.
    Tell me the sequence of musical notes shown in the image.
    """,
}

# Models
models = {
    #"Gemma3-27B": "google/gemma-3-27b-it",
    "Qwen2.5-7B": "Qwen/Qwen2.5-VL-7B-Instruct"
}

# Ensure the directory exists
os.makedirs("image_analysis", exist_ok=True)

# Create the batch job file with the prompt and content for the model and the image
def create_batch_file(model, image):
    image_url = base_url.replace("IMAGE_NAME", image)
    request = {
        "custom_id": f"image-{image}-{model}-analysis",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": models[model],
            "temperature": 0.5,
            "messages": [
                {"role": "system", "content": SYSTEM_PROMPTS[image]},
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "image_url",
                            "image_url": {"url": image_url},
                        }
                    ],
                },
            ],
        },
    }

    return request

# Save file
def save_batch_file(batch_requests, model):
    filename = f"image_analysis/batch_job_{model}_request.jsonl"
    with open(filename, "w") as file:
        for request in batch_requests:
            file.write(json.dumps(request) + "\n")
    return filename


Let's run the functions we've defined before:

In [42]:
filenames = []

for model in models:
    batch_requests = []
    for image in images:
        batch_request = create_batch_file(model, image)
        batch_requests.append(batch_request)
    filename = save_batch_file(batch_requests, model)
    filenames.append(filename)
    print(filename)

image_analysis/batch_job_Qwen2.5-7B_request.jsonl
image_analysis/batch_job_Llama3.1-8B_request.jsonl


Next, we can preview what one of the batch job files looks like:

In [43]:
!head -n 1 image_analysis/batch_job_Gemma3_request.jsonl

{"custom_id": "image-balls-image.jpg-Gemma3-analysis", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "google/gemma-3-27b-it", "temperature": 0.5, "messages": [{"role": "system", "content": "\n    You are a helpful assistant that analyzes image content.\n    Tell me the device depicted in the image, and how many balls it has.\n    "}, {"role": "user", "content": [{"type": "image_url", "image_url": {"url": "https://github.com/kluster-ai/docs/blob/main/images/tutorials/balls-image.jpg?raw=true"}}]}]}}


### Upload inference file to kluster.ai

Now that we've prepared our input files, it's time to upload it to the kluster.ai platform. To do so, you can use the `files.create` endpoint of the client, where the purpose is set to `batch`. This will return the file ID, which we need to log for the next steps. We will repeat the process for each batch file created.

In [44]:
def upload_batch_file(data_dir):
  print(f"Creating request for {data_dir}")
  
  with open(data_dir, 'rb') as file:
    upload_response = client.files.create(
    file=file,
    purpose="batch"
  )

  # Print job ID
  file_id = upload_response.id
  print(f"File uploaded successfully. File ID: {file_id}")

  return upload_response

In [45]:
batch_files = []

# Loop through all .jsonl files in the data folder
for data_dir in filenames:
    print(f"Uploading file {data_dir}")
    job = upload_batch_file(data_dir)
    batch_files.append(job)

Uploading file image_analysis/batch_job_Qwen2.5-7B_request.jsonl
Creating request for image_analysis/batch_job_Qwen2.5-7B_request.jsonl
File uploaded successfully. File ID: 67e5b8dc1e8d51bcc13081c5
Uploading file image_analysis/batch_job_Llama3.1-8B_request.jsonl
Creating request for image_analysis/batch_job_Llama3.1-8B_request.jsonl
File uploaded successfully. File ID: 67e5b8dce30eb5d129196eb8


All files are now uploaded, and we can proceed with creating the batch jobs.

### Start the job

Once all the files have been successfully uploaded, we're ready to start (create) the batch jobs by providing the file ID of each file, which we got in the previous step. To start each job, we use the `batches.create` method, for which we need to set the endpoint to `/v1/chat/completions`. This will return each batch job details, with each ID.

In [46]:
# Create batch job with completions endpoint
def create_batch_job(file_id):
  batch_job = client.batches.create(
    input_file_id=file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h"
  )

  print(f"Batch job created with ID {batch_job.id}")
  return batch_job

In [47]:
batch_jobs = []

# Loop through all batch files ID and start each job
for batch_file in batch_files:
    print(f"Creating batch job for file ID {batch_file.id}")
    batch_job = create_batch_job(batch_file.id)
    batch_jobs.append(batch_job)

Creating batch job for file ID 67e5b8dc1e8d51bcc13081c5
Batch job created with ID 67e5b8def5c2f4279cdffc5a
Creating batch job for file ID 67e5b8dce30eb5d129196eb8
Batch job created with ID 67e5b8e81e8d51bcc130826e


All requests are queued to be processed.

### Check job progress

Now that your batch jobs have been created, you can track their progress.

To monitor the job's progress, we can use the `batches.retrieve` method and pass the batch job ID. The response contains a `status` field that tells us if it is completed or not and the subsequent status of each job separately. We can repeat this process for every batch job ID we got in the previous step.

The following snippet checks the status of all batch jobs every 10 seconds until the entire batch is completed.

In [48]:
def monitor_batch_jobs(batch_jobs):
    all_completed = False

    # Loop until all jobs are completed
    while not all_completed:
        all_completed = True
        output_lines = []

        # Loop through all batch jobs
        for job in batch_jobs:
            updated_job = client.batches.retrieve(job.id)
            status = updated_job.status

            # If job is completed
            if status == "completed":
                output_lines.append("Job completed!")
            # If job failed, cancelled or expired
            elif status in ["failed", "cancelled", "expired"]:
                output_lines.append(f"Job ended with status: {status}")
                break
            # If job is ongoing
            else:
                all_completed = False
                completed = updated_job.request_counts.completed
                total = updated_job.request_counts.total
                output_lines.append(
                    f"Job status: {status} - Progress: {completed}/{total}"
                )

        # Clear terminal
        clear_output(wait=True)
        for line in output_lines:
            display(line)

        # Check every 10 seconds
        if not all_completed:
            time.sleep(10)


In [49]:
monitor_batch_jobs(batch_jobs)

'Job completed!'

'Job ended with status: failed'

## Get the results

With all jobs completed, we'll retrieve the results and review the responses generated for each request. The results are parsed. To fetch the results from the platform, you need to retrieve the `output_file_id` from the batch job, and then use the `files.content` endpoint, providing that specific file ID. We will repeat this for every single batch job id. Note that the job status must be `completed` for you to retrieve the results!

In [28]:
#Parse results as a JSON object
def parse_json_objects(data_string):
  if isinstance(data_string, bytes):
    data_string = data_string.decode('utf-8')

  json_strings = data_string.strip().split('\n')
  json_objects = []

  for json_str in json_strings:
    try:
      json_obj = json.loads(json_str)
      json_objects.append(json_obj)
    except json.JSONDecodeError as e:
      print(f"Error parsing JSON: {e}")

  return json_objects

In [29]:
# Go through all batch jobs, providing the output file ID
for batch_job in batch_jobs:
  job_status = client.batches.retrieve(batch_job.id)
  result_file_id = job_status.output_file_id
  result = client.files.content(result_file_id).content
  results = parse_json_objects(result)

  # For each, print the result
  for res in results:
    inference_id = res['custom_id']
    result = res['response']['body']['choices'][0]['message']['content']
    print(f'\n -------------------------- \n')
    print(f"{res['custom_id']}\n\nRESULT: {result}")


 -------------------------- 

image-balls-image.jpg-Qwen2.5-7B-analysis

RESULT: The image appears to be a grayscale pattern with no discernible objects or devices. It looks like a texture or a random pattern rather than a depiction of a specific device or object. There are no balls visible in this image.

 -------------------------- 

image-eggs-image.jpg-Qwen2.5-7B-analysis

RESULT: The image you provided does not contain any visible eggs. It appears to be a patterned background with no discernible objects or items that resemble eggs. Therefore, it is not possible to count the number of eggs or categorize them by color based on this image.

 -------------------------- 

image-music-image.jpeg-Qwen2.5-7B-analysis

RESULT: The image you provided does not contain any musical notes. It appears to be an abstract pattern with black and white lines on a dark background, which does not represent musical notation. If you have a different image or need assistance with something else, please l

## Summary

This tutorial used the chat completion endpoint to image analysis on multiple images using kluster.ai batch API. This particular example uploaded three specific images:

1. A <a href="https://github.com/kluster-ai/docs/blob/main/images/tutorials/balls-image.jpg?raw=true" target="_blank">Newtown's cradle</a>, we asked what is the device's name and how many balls were in the image (5)
2. <a href="https://github.com/kluster-ai/docs/blob/main/images/tutorials/eggs-image.jpg?raw=true" target="_blank">Eggs</a> of different colors, asked how many total eggs and per color (10 total, 8 brown and 2 white)
3. <a href="https://github.com/kluster-ai/docs/blob/main/images/tutorials/music-image.jpeg?raw=true" target="_blank">Hand written musical note</a>, we asked what is the sequence of notes depicted (C, D, E and F)

Both models performed....

To submit a batch job, we've:

1. Created the JSONL file, where each line of the file represented a separate request
2. Submitted the file to the platform
3. Started the batch job, and monitored its progress
4. Once completed, we fetched the results

All of this using the OpenAI Python library and API, no changes needed!

Kluster.ai's batch API empowers you to scale your workflows seamlessly, making it an invaluable tool for processing extensive datasets. As next steps, feel free to create your own dataset, or expand on top of this existing example. Good luck!